# What happens:
File is saved and ready to be uploaded to RIC

## Import 

In [1]:
import os
import sys
import pandas as pd
import numpy as np
from pandas import DataFrame
import sqlalchemy
from sqlalchemy import create_engine
import getpass
import datetime
import win32com.client as win32
import calendar
from openpyxl import Workbook, load_workbook
from IPython.lib.display import Audio

### Select the folder where you wanna do the work
### Also ask for the Reporting Date

In [2]:
directory = os.path.abspath(os.getcwd()) + "/"
os.chdir(directory)

rd = str(input("Please enter the RD (YYYYMMDD): "))

ws_cons = "0002"
ws_uncons = "0194"

### Make connection with Oracle DB

In [3]:
#Setting up RAY connection:
connection_RAY = create_engine(str("oracle://:@OCRPN"))
connection_RAY.execute("call pack_context.context_open(to_date('{reporting_date}','YYYYMMDD'),{partition})".format(
                 reporting_date = rd, partition = ws_uncons))

### Select securities

In [4]:
%%time
d1 = pd.read_sql_query("""select s.security_Reference as security, cr.rating_code, r.book_code, r.attribute_5, r.repo_Type, r.contract_type, r.counterparty_code, e.entity_desc 
from security s 
left join repo_portfolio rp on s.security_reference = rp.security_reference 
left join repo r on rp.repo_reference = r.contract_Reference 
left join entity e on r.counterparty_code = e.entity_code 
left join credit_Rating cr on s.security_reference = cr.reference 
where s.security_reference like ('TRI_%') 
and r.attribute_5 is not null """, connection_RAY)

CPU times: total: 0 ns
Wall time: 647 ms


In [5]:
d1_final = d1.drop_duplicates(subset=['security'])
d1_final

,security,rating_code,book_code,attribute_5,repo_type,contract_type,counterparty_code,entity_desc
0,TRI_BNPA_P7,None,HO_T,A961693*ARTS*ARTS*HO,RR,IL0101,161038,BNP Paribas Arbitrage SNC


### Open MC Dummy ISINs in ARTS all

In [8]:
im_folder = "Import/"
ex_folder = "Export/"
dummy_isin = pd.read_excel(im_folder + "MC Dummy ISINs in ARTS all.xlsx")
triparty = load_workbook(im_folder + "CW_check - Template.xlsx")

credit_rating = triparty['DataInput2']
non_rating_list = ['ACTIVE', 'FUNDS', 'MAIN INDEX', 'NON MAIN INDEX']

d1_final.security.tolist()
d1_final
added = 0
date = "31.01.2015 00:00"
dummy_isin
for i, security in enumerate(d1_final.security.tolist()):
    if security in dummy_isin.REFERENCE.tolist():
#         print(security)
#         print(dummy_isin.REFERENCE.tolist().index(security))
#         print(dummy_isin.RATING_CODE[dummy_isin.REFERENCE.tolist().index(security)])
        if dummy_isin.RATING_CODE[dummy_isin.REFERENCE.tolist().index(security)] not in non_rating_list:
            credit_rating.cell(row = added + 19, column = 1).value = security
            credit_rating.cell(row = added + 19, column = 2).value = "STD_POORS"
            credit_rating.cell(row = added + 19, column = 3).value = date
            credit_rating.cell(row = added + 19, column = 4).value = "EQUITY"
            credit_rating.cell(row = added + 19, column = 5).value = dummy_isin.RATING_CODE[dummy_isin.REFERENCE.tolist().index(security)]
            added =+ 1
    else:
        print(security + " does not exist")
    
triparty.save(ex_folder + "CREDIT_RATING_TRIparty_" + rd + ".xlsx")